In [3]:
!pip install flask

  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
from flask import Flask, request, render_template
import pickle
import pandas as pd
import socket

app = Flask(__name__)

# Load trained pipeline
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

@app.route('/')
def home():
   # Get unique test types from your dataset
    test_options = df['language_proficiency_test'].dropna().unique().tolist()
    return render_template('index.html', test_options=test_options)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        gpa_or_score = float(request.form['gpa_or_score'])
        language_proficiency_test = request.form['language_proficiency_test']
        test_score = float(request.form['test_score'])

        # Prepare DataFrame
        input_df = pd.DataFrame({
            'gpa_or_score': [gpa_or_score],
            'language_proficiency_test': [language_proficiency_test],
            'test_score': [test_score]
        })

        prediction = model.predict(input_df)[0]

        return render_template('index.html', prediction_text=f"Predicted Placement Status: {prediction}")
    except Exception as e:
        return str(e)

def find_free_port(start_port=5000, max_port=5100):
    """Finds a free port between start_port and max_port"""
    for port in range(start_port, max_port):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('127.0.0.1', port)) != 0:
                return port
    return start_port  # fallback

if __name__ == '__main__':
    port = find_free_port()
    print(f"Starting Flask app on http://127.0.0.1:{port}")
    # Set debug=True for development; set to False in production
    app.run(debug=True, port=port, use_reloader=False)

Starting Flask app on http://127.0.0.1:5001
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [13/Aug/2025 21:25:47] "GET / HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/Users/rushita/Desktop/venv/lib/python3.12/site-packages/flask/app.py", line 1536, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rushita/Desktop/venv/lib/python3.12/site-packages/flask/app.py", line 1514, in wsgi_app
    response = self.handle_exception(e)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rushita/Desktop/venv/lib/python3.12/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rushita/Desktop/venv/lib/python3.12/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rushita/Desktop/venv/lib/python3.12/site-packages/flask/app.py"

In [ ]:
!python app.py